In [1]:
%load_ext autoreload
%autoreload 2
from datasets import *
import re

py_data = load_dataset("bigcode/the-stack", split="train", data_files="data/python/train-00000-of-00206.parquet")
java_data = load_dataset("bigcode/the-stack", split="train", data_files="data/java/train-00005-of-00285.parquet")

/home/franlucc/mechinterp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset parquet (/home/franlucc/.cache/huggingface/datasets/bigcode___parquet/bigcode--the-stack-feef6e400b84213f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (/home/franlucc/.cache/huggingface/datasets/bigcode___parquet/bigcode--the-stack-9f45e66b86f2b01e/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [16]:
selected_py_data = [i for i in py_data[:100]["content"] if bool(re.search(r'\d', i))]
selected_java_data = [i for i in java_data[:100]["content"] if bool(re.search(r'int ', i)) and bool(re.search(r'\d', i))]

In [17]:
def crop_int_function(selected_data):
    edited_data = {"python":[]}
    for ex in selected_data:
        # cut out function with int
        ints = re.findall(r'\n\n *def[\w\W]*?:[\w\W]*?\d[\w\W]*?\n', ex)
        unrolled = [i.split("\n\n")[-1] for i in ints]
        edited_data["python"] += unrolled
    return Dataset.from_dict(edited_data)

py_data = crop_int_function(selected_py_data)

In [26]:
len(py_data)

py_data.save_to_disk("../data/python_int_funcs.parquet")

Saving the dataset (0/1 shards):   0%|          | 0/399 [00:00<?, ? examples/s]

In [27]:
len(load_from_disk("../data/python_int_funcs.parquet"))

399

In [2]:
dataset = load_dataset("nuprl/stack-dedup-python-fns-returns-typechecks", split="train")

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 750.86it/s]


Dataset parquet downloaded and prepared to /home/franlucc/.cache/huggingface/datasets/nuprl___parquet/nuprl--stack-dedup-python-fns-returns-typechecks-4eeb9fc26e67528b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


In [43]:
dataset = load_dataset("nuprl/stack-dedup-python-testgen-starcoder", split="train")

Found cached dataset parquet (/home/franlucc/.cache/huggingface/datasets/nuprl___parquet/nuprl--stack-dedup-python-testgen-starcoder-3806945f8dbcae94/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [3]:
int_data = {"examples" : []}
for ex in dataset["content"]:
    if [] != re.findall(r': int ', ex):
        int_data["examples"].append(ex)

In [8]:
print(int_data["examples"][0])

def mover_alfil(tablero, x_inicial, y_inicial, x_final, y_final):
    """
    (list of list, int, int, int, int) -> list of list

    :param tablero: list of list que representa el tablero
    :param x_inicial: int que representa la posicion inicial en X
    :param y_inicial: int que representa la posicion inicial en Y
    :param x_final: int que representa la posicion final en X
    :param y_final: int que representa la posicion final en Y
    :return: list of list que representa un tablero final
    """
    tab = tablero.copy()
    if ((x_inicial - y_inicial == x_final - y_final) or (x_inicial + y_inicial == x_final + y_final)) and tab[x_inicial][y_final].lower() == 'a':
        if (x_inicial != x_final) and (y_inicial != y_final):
            for x in range(x_inicial +1, x_final):
                if tab[x][y_final] != ' ':
                    raise ValueError('El camino no es valido')
            for y in range(y_inicial +1, y_final):
                if tab[x_final][y] != ' ':
     

In [9]:
int_data = Dataset.from_dict(int_data)
int_data.save_to_disk("../data/py_int_var_funcs_large.parquet")